In [ ]:
#! pip install fairlearn
#! pip install lightgbm
#! pip install optuna

In [1]:
from sklearn.metrics import f1_score, confusion_matrix, make_scorer, accuracy_score
from sklearn.model_selection import cross_val_score, train_test_split, cross_validate
from fairlearn.metrics import (
    count,
    selection_rate,
    equalized_odds_difference,
    false_positive_rate,
    false_negative_rate,
    demographic_parity_difference
)

from fairlearn.datasets import fetch_adult
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.compose import make_column_selector as selector
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestClassifier
from lightgbm import LGBMClassifier

import numpy as np
import optuna
import tqdm as notebook_tqdm

/home/azucena/miniconda3/envs/fairenv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [30]:
data = fetch_adult(as_frame=True)
X_raw = data.data
y = (data.target == ">50K") * 1
A = X_raw["sex"]

numeric_transformer = Pipeline(
    steps=[
        ("impute", SimpleImputer()),
        ("scaler", StandardScaler()),
    ]
)
categorical_transformer = Pipeline(
    [
        ("impute", SimpleImputer(strategy="most_frequent")),
        ("ohe", OneHotEncoder(handle_unknown="ignore")),
    ]
)
preprocessor = ColumnTransformer(
    transformers=[
        ("num", numeric_transformer, selector(dtype_exclude="category")),
        ("cat", categorical_transformer, selector(dtype_include="category")),
    ]
)

pipeline = Pipeline(
    steps=[
        ("preprocessor", preprocessor),
        (
            "classifier",
            LGBMClassifier(n_jobs=-1),
        ),
    ]
)

/home/azucena/miniconda3/envs/fairenv/lib/python3.10/site-packages/sklearn/datasets/_openml.py:968: FutureWarning:

The default value of `parser` will change from `'liac-arff'` to `'auto'` in 1.4. You can set `parser='auto'` to silence this warning. Therefore, an `ImportError` will be raised from 1.4 if the dataset is dense and pandas is not installed. Note that the pandas parser may return different data types. See the Notes Section in fetch_openml's API doc for details.



In [31]:
def metric_scorer(clf, X, y):
    y_pred = clf.predict(X)
    f1 = f1_score(y,y_pred)
    abs_eod = np.abs(equalized_odds_difference(y, y_pred, sensitive_features=X['sex']))
    return {'f1_score': f1, 'eod': abs_eod}

def metric_scorer_v2(clf, X, y):
    y_pred = clf.predict(X)
    error = 1 - accuracy_score(y, y_pred)
    abs_dpd = np.abs(demographic_parity_difference(y, y_pred, sensitive_features=X['sex']))
    return {'error': error, 'dpd': abs_dpd}

def metric_scorer_v3(clf, X, y):
    y_pred = clf.predict(X)
    f1 = f1_score(y,y_pred)
    abs_dpd = np.abs(demographic_parity_difference(y, y_pred, sensitive_features=X['sex']))
    return {'f1_score': f1, 'dpd': abs_dpd}


In [4]:
from fairlearn.metrics import equalized_odds_difference
import numpy as np

pipeline.fit(X_train, y_train)
y_pred = pipeline.predict(X_test)
#fair_value = np.abs(equalized_odds_difference(y_test, y_pred, sensitive_features=A_test))
fair_value = np.abs(demographic_parity_difference(y_test, y_pred, sensitive_features=A_test))
model_value = f1_score(y_test, y_pred) 

print(fair_value)
print(model_value)

0.1908097330743628
0.6951638780655514


In [33]:
seed = 128
num_variables = 2
n_startup_trials = 11 * num_variables - 1
directions = ['minimize', 'maximize']
results = []

for sim in [0,1,2,3]:
    print(sim)
    def objective(trial):

        (X_train, X_test, y_train, y_test, A_train, A_test) = train_test_split(
        X_raw, y, A, test_size=0.8, random_state=sim, stratify=y
        )

        X_train = X_train.reset_index(drop=True)
        X_test = X_test.reset_index(drop=True)
        y_train = y_train.reset_index(drop=True)
        y_test = y_test.reset_index(drop=True)
        A_train = A_train.reset_index(drop=True)
        A_test = A_test.reset_index(drop=True)


        params = {
            'n_estimators': trial.suggest_int("n_estimators", 20, 10000),
            'num_leaves': trial.suggest_int("num_leaves", 10, 1000),
            'max_depth': trial.suggest_int("max_depth", 2, 20),
            'min_child_samples': trial.suggest_int("min_child_samples", 5, 300),
            'learning_rate': trial.suggest_float('learning_rate', 1e-5, 1e-2),
            'boosting_type': trial.suggest_categorical("boosting_type", ['goss', 'gbdt'])
            }
        pipeline['classifier'].set_params(**params)

        scores = cross_validate(
                pipeline, 
                X_train,
                y_train, 
                cv=5,
                scoring = metric_scorer_v3,
                return_train_score=True)

        #fair_metric = scores['test_eod'].mean()
        fair_metric = scores['test_dpd'].mean()
        model_metric = scores['test_f1_score'].mean()

        return fair_metric, model_metric
    
    #sampler = optuna.samplers.MOTPESampler(
    #    n_startup_trials=n_startup_trials, n_ehvi_candidates=24, seed=seed
    #    )
    sampler = optuna.samplers.TPESampler()  # `MOTPESampler` used to be required for multi-objective optimization.
    study = optuna.create_study(directions=directions, pruner=optuna.pruners.SuccessiveHalvingPruner(), sampler = sampler)
    study.optimize(objective, n_trials=100)
    print("Number of finished trials: ", len(study.trials))
    results.append(study)
    #return fair_metric

/tmp/ipykernel_78827/2727072634.py:47: FutureWarning:

MOTPESampler has been deprecated in v2.9.0. This feature will be removed in v4.0.0. See https://github.com/optuna/optuna/releases/tag/v2.9.0.

[I 2023-05-09 19:21:30,298] A new study created in memory with name: no-name-7f9a429f-8c78-452c-8992-017201f88e12


0


[I 2023-05-09 19:22:01,137] Trial 0 finished with values: [0.1611839665808317, 0.6940610139842006] and parameters: {'n_estimators': 8669, 'num_leaves': 270, 'max_depth': 4, 'min_child_samples': 17, 'learning_rate': 0.0023968541035582405, 'boosting_type': 'gbdt'}. 
[I 2023-05-09 19:22:26,430] Trial 1 finished with values: [0.1582795779038875, 0.6651464111737642] and parameters: {'n_estimators': 6022, 'num_leaves': 341, 'max_depth': 4, 'min_child_samples': 96, 'learning_rate': 0.001106140614373575, 'boosting_type': 'goss'}. 
[I 2023-05-09 19:22:49,830] Trial 2 finished with values: [0.1501082611015477, 0.6516546528493607] and parameters: {'n_estimators': 3011, 'num_leaves': 522, 'max_depth': 15, 'min_child_samples': 280, 'learning_rate': 0.0010298606581265368, 'boosting_type': 'gbdt'}. 
[I 2023-05-09 19:23:26,755] Trial 3 finished with values: [0.18512677216320367, 0.6277775171173711] and parameters: {'n_estimators': 5600, 'num_leaves': 596, 'max_depth': 16, 'min_child_samples': 248, 'le

Number of finished trials:  100
1


[I 2023-05-09 20:08:41,519] Trial 0 finished with values: [0.18623644358378555, 0.6916948547239662] and parameters: {'n_estimators': 8669, 'num_leaves': 270, 'max_depth': 4, 'min_child_samples': 17, 'learning_rate': 0.0023968541035582405, 'boosting_type': 'gbdt'}. 
[I 2023-05-09 20:09:10,115] Trial 1 finished with values: [0.1832638675273926, 0.6652062662491863] and parameters: {'n_estimators': 6022, 'num_leaves': 341, 'max_depth': 4, 'min_child_samples': 96, 'learning_rate': 0.001106140614373575, 'boosting_type': 'goss'}. 
[I 2023-05-09 20:09:31,330] Trial 2 finished with values: [0.1812502577378719, 0.6534469461242683] and parameters: {'n_estimators': 3011, 'num_leaves': 522, 'max_depth': 15, 'min_child_samples': 280, 'learning_rate': 0.0010298606581265368, 'boosting_type': 'gbdt'}. 
[I 2023-05-09 20:10:02,391] Trial 3 finished with values: [0.19872452430800355, 0.6418934718946584] and parameters: {'n_estimators': 5600, 'num_leaves': 596, 'max_depth': 16, 'min_child_samples': 248, 'l

Number of finished trials:  100
2


[I 2023-05-09 20:58:48,665] Trial 0 finished with values: [0.18981368447257033, 0.6961217811573337] and parameters: {'n_estimators': 8669, 'num_leaves': 270, 'max_depth': 4, 'min_child_samples': 17, 'learning_rate': 0.0023968541035582405, 'boosting_type': 'gbdt'}. 
[I 2023-05-09 20:59:18,439] Trial 1 finished with values: [0.19245392516760246, 0.6771095141290172] and parameters: {'n_estimators': 6022, 'num_leaves': 341, 'max_depth': 4, 'min_child_samples': 96, 'learning_rate': 0.001106140614373575, 'boosting_type': 'goss'}. 
[I 2023-05-09 20:59:42,566] Trial 2 finished with values: [0.18998066729013785, 0.6636559099508768] and parameters: {'n_estimators': 3011, 'num_leaves': 522, 'max_depth': 15, 'min_child_samples': 280, 'learning_rate': 0.0010298606581265368, 'boosting_type': 'gbdt'}. 
[I 2023-05-09 21:00:18,179] Trial 3 finished with values: [0.216517816362445, 0.6403236649325972] and parameters: {'n_estimators': 5600, 'num_leaves': 596, 'max_depth': 16, 'min_child_samples': 248, 'l

Number of finished trials:  100
3


[I 2023-05-09 22:32:09,103] Trial 0 finished with values: [0.18262568538012144, 0.7036810097971111] and parameters: {'n_estimators': 8669, 'num_leaves': 270, 'max_depth': 4, 'min_child_samples': 17, 'learning_rate': 0.0023968541035582405, 'boosting_type': 'gbdt'}. 
[I 2023-05-09 22:32:41,707] Trial 1 finished with values: [0.18760185283406314, 0.675392259179757] and parameters: {'n_estimators': 6022, 'num_leaves': 341, 'max_depth': 4, 'min_child_samples': 96, 'learning_rate': 0.001106140614373575, 'boosting_type': 'goss'}. 
[I 2023-05-09 22:33:06,122] Trial 2 finished with values: [0.18182563897828635, 0.6694234991309532] and parameters: {'n_estimators': 3011, 'num_leaves': 522, 'max_depth': 15, 'min_child_samples': 280, 'learning_rate': 0.0010298606581265368, 'boosting_type': 'gbdt'}. 
[I 2023-05-09 22:33:42,594] Trial 3 finished with values: [0.2067847398168028, 0.6441838099474957] and parameters: {'n_estimators': 5600, 'num_leaves': 596, 'max_depth': 16, 'min_child_samples': 248, 'l

Number of finished trials:  100


In [32]:
#directions = ['minimize', 'maximize']
#directions = 'minimize'
#study = optuna.create_study(
#    direction=directions, 
#    pruner=optuna.pruners.SuccessiveHalvingPruner(), 
#    sampler=optuna.samplers.TPESampler()
#    )


#sampler = optuna.samplers.MOTPESampler(
#    n_startup_trials=n_startup_trials, n_ehvi_candidates=24, seed=seed
#)
#study = optuna.create_study(directions=directions, pruner=optuna.pruners.SuccessiveHalvingPruner(), sampler = sampler)
#study.optimize(objective, n_trials=100)

#print("Number of finished trials: ", len(study.trials))

In [35]:
import dill
file_name = 'f1-dpd-lgbm-motpe-succesivehalving-100trials-4sim.pkl'
#f1-eod-lgbm-succesivehalving-30trails.pkl
with open(file_name, 'wb') as file:
    dill.dump(results, file)
    print(f'Object successfully saved to "{file_name}"')

Object successfully saved to "f1-dpd-lgbm-motpe-succesivehalving-100trials-4sim.pkl"


In [29]:
optuna.visualization.plot_pareto_front(study, target_names=["fair_metric", "f1_score"], include_dominated_trials = True)

In [8]:
#print(study.best_params)
#print(study.best_value)

In [21]:
#best_params = {name.replace("lgbm_",""):param for name,param in study.best_params.items()}
#pipeline['classifier'].set_params(**best_params)
#pipeline.fit(X_train, y_train)
#y_pred = pipeline.predict(X_test)

In [22]:
#from fairlearn.metrics import equalized_odds_difference
#import numpy as np

#def objective_function(fairness_metric, model_metric, y_test, y_pred, sensitive_features, alpha = .5):
#fair_value = np.abs(equalized_odds_difference(y_test, y_pred, sensitive_features=A_test))
#fair_value = np.abs(demographic_parity_difference(y_test, y_pred, sensitive_features=A_test))
#model_value = f1_score(y_test, y_pred) 

#print(fair_value)
#print(model_value)

0.0
0.0
